In [96]:
# Import dependencies
import bit
import lit
from web3 import Web3
from eth_account import Account
import subprocess
import json
import os
from dotenv import load_dotenv
from constants import *

In [97]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

In [98]:
cols = 'address,index,path,privkey,pubkey,pubkeyhash,xprv,xpub'

In [111]:
# Create a function called `derive_wallets`
def derive_wallets(coin):
    command = f'derive --mnemonic=mnemonic --cols={cols} --coin={coin} --numderive=12 --format=json -g'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [112]:
coins = {BTCTEST: derive_wallets(BTCTEST), ETH: derive_wallets(ETH), BTC: derive_wallets(BTC), LTC: derive_wallets(LTC)}

In [113]:
coins

{'btc-test': [{'address': 'mtgb9hthmFpY1zidrYmZ24eTF87aBA147h',
   'index': 0,
   'path': "m/44'/1'/0'/0/0",
   'privkey': 'cVAgDoahzo5Ucdv8ioxxubvmE6LfU3ZQgJ21UvY4eC1xi6WkvnXe',
   'pubkey': '0339b281b12687c70a024ef409229b55d404533b4e9a475dc9d3883c0172913fb6',
   'pubkeyhash': '906c250da82c2bd3edf9bcbbbb29f2188065f9d2',
   'xprv': 'tprv8kmimjp9GYq9EvcrxozEhxzk9jGwaC73BSiy3ATqwwikM4YgH7685zQLzX7xzXAmhXxf5LZbpbqtSGX3fcNFbfVJdxuEYPrA5tDRRQgwPRm',
   'xpub': 'tpubDHTkv9rPQvWp8PeerTeq7NeriknsjXHwkkKkKgW9NDX9BYoSuVuiGV2DAejLNv85zVm31akUTVk6Z8MyhECbWDzQ3Qir4GoRCXfBgKk8GxA'},
  {'address': 'mxwSU31v1NtbUi6Pc5bN9BgmmAGw13JdvX',
   'index': 1,
   'path': "m/44'/1'/0'/0/1",
   'privkey': 'cRG5vrAkkq3ZkTmT2mmqCgbxd8niBdXx3ocDMd1771oSzJDvqXiC',
   'pubkey': '02f4468eae06b195a4f83cc63d680d015d512af60288977e0283fd4299bfcde24b',
   'pubkeyhash': 'bf1ba366dd65a4824c22f497bbff1fae89007841',
   'xprv': 'tprv8kmimjp9GYq9FvSyaAKU2MY38GBpiMHEsjgMtnLaMhyZQPzAiEM6ZTC8z9vdTsJVRQDcoCvLeWEHiPWWLpiGMjoYcffLhXJXN

In [107]:
coins['eth'][0]['privkey']

'0x82d63adc8be924a92ef77c6e3286c4e80eaebad63ba2b91c27801a62ba2f9410'

In [114]:
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    elif coin == BTCTEST:
        return bit.PrivateKeyTestnet(priv_key)
#     elif coin == LTC:
#         return lit.PrivateKeyTestnet(priv_key)

In [115]:
priv_key_to_account(ETH, coins[ETH][0]['privkey'])

In [119]:
def create_tx(coin, account, recipient, amount):
    if coin == ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "chainId": 987,
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == BTCTEST:
        return bit.PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])